In [2]:
from datetime import datetime, timedelta
import uuid
from ortools.sat.python import cp_model
import pandas as pd
import pickle
import csv

In [11]:
class Schedule:


    def __init__(self, name: str, start_time: datetime, end_time: datetime):
        assert start_time < end_time, 'Start time must be before end time'
        self.name = name
        self.start_time = start_time
        self.end_time = end_time
        self.employees = []
        self.shifts = []

        self._holidays = self.get_weekends(start_time, end_time)

        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()
        
        
    
    @property
    def dates(self) -> list[datetime]:
        return [self.start_time + timedelta(days=x) for x in range(self.duration.days + 1)]

    @property
    def duration(self) -> timedelta:
        return self.end_time - self.start_time

    @property
    def holidays(self) -> list[datetime]:
        return self._holidays

    @property
    def roles(self) -> set:
        return(set([employee.role for employee in self.employees]))

    @property
    def shift_types(self) -> set:
        return set([shift.shift_type for shift in self.shifts])


    def add_holiday(self, date: datetime) -> None:
        # Check if date is in dates 
        if date not in self.dates:
            raise ValueError('Date is not in schedule')
        # Check if date is already a holiday
        if date in self._holidays:
            raise ValueError('Date is already a holiday')
        self._holidays.append(date)
        self.__updated_at = datetime.now()

    def remove_holiday(self, date: datetime) -> None:
        # Check if date is in holidays
        if date in self._holidays:
            self._holidays.remove(date)
            self.__updated_at = datetime.now()
        else:
            raise ValueError('Date is not a holiday')

    @staticmethod
    def get_weekends(start_time: datetime, end_time: datetime) -> list[datetime]:
        return [start_time + timedelta(days=x) for x in range((end_time - start_time).days + 1) if (start_time + timedelta(days=x)).weekday() in [5, 6]]

    def __repr__(self) -> str:
        return f'{self.name} From {self.start_time.isoformat()} to {self.end_time.isoformat()}'

    def __str__(self) -> str:
        return f'{self.name} From {self.start_time.isoformat()} to {self.end_time.isoformat()}'

    def reset(self) -> None:
        self.employees = []
        self.shifts = []

 
    def add_employee(self, employee) -> object:
        self.employees.append(employee)
        self.__updated_at = datetime.now()
        return employee

    def add_shift(self, shift) -> object:
        self.shifts.append(shift)
        self.__updated_at = datetime.now()
        return shift


    def remove_employee(self, employee) -> None:
        self.employees.remove(employee)
        self.__updated_at = datetime.now()

    def remove_shift(self, shift) -> None:
        self.shifts.remove(shift)
        self.__updated_at = datetime.now()

    def show(self, format = 'text', group_by = 'shift type') -> None:        
        
        if format == 'text':
            self.__display_text()
        elif format == 'table':
            self.__display_table(group_by=group_by)
        else:
            raise Exception('Invalid format')
    

    def __display_text(self) -> None:
        print('Schedule Name: {}'.format(self.name))
        print('Start Time: {}'.format(self.start_time))
        print('End Time: {}'.format(self.end_time))
        print('Duration: {}'.format(self.duration))
        print('Employees: {}'.format(self.employees))
        print('Shifts: {}'.format(self.shifts))
        print('Created At: {}'.format(self.__created_at))
        print('Updated At: {}'.format(self.__updated_at))

    def __display_table(self, group_by = 'shift type') -> pd.DataFrame:
        dates = [date.date() for date in pd.date_range(self.start_time, self.end_time, freq='D')]
        shifts = self.shifts
        shift_types = self.shift_types
        employees = self.employees

        if group_by == 'shift':
            # Create a dataframe with the dates as the index and the shifts as the columns
            shift_schedule = pd.DataFrame(index=dates, columns=[shift.name for shift in shifts])

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'None'
            # If a shift is assigned to multiple employees, fill with list of employees
            # If there're no shifts on a given day, fill with "-"
            for date in dates:
                for shift in shifts:
                    if shift.start_time.date() == date:
                        if len(shift.employees) == 0:
                            shift_schedule.loc[date, shift.name] = 'None'
                        elif len(shift.employees) == 1:
                            shift_schedule.loc[date, shift.name] = shift.employees[0].first_name
                        else:
                            shift_schedule.loc[date, shift.name] = [employee.first_name for employee in shift.employees]
                    else:
                        shift_schedule.loc[date, shift.name] = '-'
        elif group_by == 'shift type':
            # Create a dataframe with the dates as the index and the shift types as the columns
            shift_schedule = pd.DataFrame(index=dates, columns=[shift_type for shift_type in shift_types])

            # Fill the dataframe with the employees assigned to each shift, if no shift is assigned, fill with 'None'
            # If a shift is assigned to multiple employees, fill with list of employees
            # If there're no shifts on a given day, fill with "-"
            # Group the shifts by shift type as the columns
            shift_by_type = {}
            for shift in shifts:
                if shift.shift_type in shift_by_type:
                    shift_by_type[shift.shift_type].append(shift)
                else:
                    shift_by_type[shift.shift_type] = [shift]

            for shift_type in shift_by_type:
                for shift in shift_by_type[shift_type]:
                    for date in dates:
                        if shift.start_time.date() == date:
                            if len(shift.employees) == 0:
                                shift_schedule.loc[date, shift_type] = 'Unassigned'
                            elif len(shift.employees) == 1:
                                shift_schedule.loc[date, shift_type] = shift.employees[0].first_name
                            else:
                                shift_schedule.loc[date, shift_type] = [employee.first_name for employee in shift.employees]
                        else:
                            # if there's no shift on a given day, fill with '-', but preserve added shifts 
                            if shift_type not in shift_schedule.columns:
                                shift_schedule[shift_type] = '-'
                    
        else:
            raise ValueError(f'Invalid value for group_by: {group_by}')
        
        
        # Display the dataframe
        display(shift_schedule)

        return shift_schedule


    # Solve the schedule with CP-SAT
    def solve(self):
        model = cp_model.CpModel()

        # Create the variables (shift, employee)
        shifts = {}
        for shift in self.shifts:
            for employee in self.employees:
                shifts[(shift, employee)] = model.NewBoolVar('shift_{}_employee_{}'.format(shift.name, employee.name))

        # Create the constraints
        # Each shift must be assigned to employees more than or equal to min_employees, and less than or equal to max_employees
        for shift in self.shifts:
            model.Add(sum(shifts[(shift, employee)] for employee in self.employees) >= shift.min_employees)
            model.Add(sum(shifts[(shift, employee)] for employee in self.employees) <= shift.max_employees)

        # Solve the model
        solver = cp_model.CpSolver()
        status = solver.Solve(model)

        # Print the solution
        if (status == cp_model.OPTIMAL) or (status == cp_model.FEASIBLE):
            for shift in self.shifts:
                for employee in self.employees:
                    if solver.Value(shifts[(shift, employee)]) == 1:
                        print('Employee {} is assigned to shift {}'.format(employee.name, shift.name))

        else:
            print('No solution found.')
            return None

        # Update the schedule with the solution
        for shift in self.shifts:
            for employee in self.employees:
                if solver.Value(shifts[(shift, employee)]) == 1:
                    shift.add_employee(employee)
                    employee.add_task(shift)

        self.__updated_at = datetime.now()

        return self

    def save(self, file_name):
        with open(file_name, 'wb') as f:
            pickle.dump(self, f)

    def to_csv(self, file_name):
        self.__display_table(group_by="shift type").to_csv(file_name)

    @staticmethod
    def load(file_name):
        with open(file_name, 'rb') as f:
            return pickle.load(f)

    


class Employee:
    def __init__(self, first_name: str, last_name: str, role: str):
        assert len(first_name) > 0, 'First name cannot be empty'
        assert len(last_name) > 0, 'Last name cannot be empty'
        assert len(role) > 0, 'Role cannot be empty'

        self.first_name = first_name
        self.last_name = last_name
        self.name = first_name + " " + last_name
        self.role = role
        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()
        self.tasks = []

    def __repr__(self) -> str:
        return f"Employee('{self.name}', '{self.role}', {self.tasks})"

    def add_task(self, task):
        # Check if task is already in the list
        if task in self.tasks:
            raise Exception('Task is already in the list')
        self.tasks.append(task)
        self.__updated_at = datetime.now()
    
    def remove_task(self, task):
        # Check if task is in the list
        if task not in self.tasks:
            raise Exception('Task is not in the list')
        self.tasks.remove(task)
        self.__updated_at = datetime.now()

    def reset_tasks(self):
        self.tasks = []
        self.__updated_at = datetime.now()

    @classmethod
    def from_csv(cls, file_name: str) -> list:
        employees = []
        with open(file_name, 'r') as f:
            reader = csv.reader(f)
            header = next(reader) # save the header for indexing
            for row in reader:
                employees.append(cls(row[header.index('first_name')], row[header.index('last_name')], row[header.index('role')]))
            
        return employees


class Task:
    def __init__(self, name: str, description: str, start_time: datetime, duration: timedelta):
        
        self.name = name
        self.description = description
        assert duration >= timedelta(minutes=0), "duration must be greater than or equal to 0"
        self.start_time = start_time
        self.duration = duration
        self.end_time = start_time + duration
        self.__id = uuid.uuid4()
        self.__created_at = datetime.now()
        self.__updated_at = datetime.now()

    def __repr__(self) -> str:
        return self.name


class Shift(Task):
    def __init__(self, name: str, description: str, duration: timedelta, start_time: datetime, shift_type: str, min_employees: int, max_employees: int):
        super().__init__(name, description, start_time, duration)
        self.shift_type = shift_type
        assert min_employees <= max_employees, "min_employees must be less than or equal to max_employees"
        assert min_employees >= 0, "min_employees must be greater than or equal to 0"
        assert max_employees >= 0, "max_employees must be greater than or equal to 0"
        self.min_employees = min_employees
        self.max_employees = max_employees
        self.employees = []
        # self.date = start_time.date()

    def add_employee(self, employee):
        # Check if employee is already in the list
        if employee in self.employees:
            raise Exception('Employee is already in the list')
        self.employees.append(employee)
        self.__updated_at = datetime.now()

    def remove_employee(self, employee):
        # Check if employee is in the list
        if employee not in self.employees:
            raise Exception('Employee is not in the list')
        self.employees.remove(employee)
        self.__updated_at = datetime.now()

    def reset_employees(self):
        self.employees = []
        self.__updated_at = datetime.now()




In [5]:
# Test importing employees from a csv file
employees = Employee.from_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/inputs/staffs.csv')

employees

[Employee('บริบูรณ์ เชนธนากิจ', 'instructor', []),
 Employee('บวร วิทยชำนาญกุล', 'executive', []),
 Employee('กรองกาญจน์ สุธรรม', 'instructor', []),
 Employee('ปริญญา เทียนวิบูลย์', 'assisted executive', []),
 Employee('ภาวิตา เลาหกุล', 'instructor', []),
 Employee('ธีรพล ตั้งสุวรรณรักษ์', 'instructor', []),
 Employee('วชิระ วงศ์ธนสารสิน', 'instructor', []),
 Employee('บุญฤทธิ์ คำทิพย์', 'instructor', []),
 Employee('ชานนท์ ช่างรัตนากร', 'instructor', []),
 Employee('กอสิน เลาหะวิสุทธิ์', 'assisted instructor', []),
 Employee('พิมพ์พรรณ อัศวสุรอิน', 'assisted instructor', []),
 Employee('ณัฐฐิกานต์ มีลาภ', 'assisted instructor', [])]

In [15]:
# create a schedule for March 2023
schedule = Schedule('March 2023', datetime(2023, 3, 1), datetime(2023, 4, 1))

# add employees
employees = Employee.from_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/inputs/staffs.csv')
for employee in employees:
    schedule.add_employee(employee)

# for i in range(15):
#     schedule.add_shift(Shift(name=f'Shift{i}', 
#                             description=f'Shift{i} description', 
#                             duration=timedelta(hours=8), 
#                             start_time=datetime(2023, 3, i+5, 15, 30), 
#                             shift_type='evening', 
#                             min_employees=1, 
#                             max_employees=3))


# add shifts (Service 1, service 2, service 1+, service 2+, morning conference, EMS, observe, พอป การบิน (AMD), แพทย์บิน(AVD))
for d in schedule.dates:
    if d.weekday() in [0, 1, 2, 3, 4]:
        # Service 1
        schedule.add_shift(Shift(name=f'Service 1 on {d.date()}',
                                description=f'Service at Emergency area from 8.00 - 12.00',
                                duration=timedelta(hours=4), 
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='service 1',
                                min_employees=2,
                                max_employees=2))
        # Service 1+
        schedule.add_shift(Shift(name=f'Service 1+ on {d.date()}',
                                description=f'Service at Urgency area from 8.00 - 12.00',
                                duration=timedelta(hours=4), 
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='service 1+',
                                min_employees=1,
                                max_employees=1))
        # Service 2
        schedule.add_shift(Shift(name=f'Service 2 on {d.date()}',
                                description=f'Service at Emergency area from 12.00 - 16.00',
                                duration=timedelta(hours=4),
                                start_time=datetime(d.year, d.month, d.day, 12, 0),
                                shift_type='service 2',
                                min_employees=1,
                                max_employees=1))
        # Service 2+
        schedule.add_shift(Shift(name=f'Service 2+ on {d.date()}',
                                description=f'Service at Urgency area from 12.00 - 16.00',
                                duration=timedelta(hours=4),
                                start_time=datetime(d.year, d.month, d.day, 12, 0),
                                shift_type='service 2+',
                                min_employees=1,
                                max_employees=1))
        # Morning conference
        schedule.add_shift(Shift(name=f'Morning conference on {d.date()}',
                                description=f'Morning conference from 8.00 - 10.00',
                                duration=timedelta(hours=2),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='morning conference',
                                min_employees=1,
                                max_employees=1))
    if d.weekday() in [0, 1, 2, 3, 4, 5, 6]:
        # EMS
        schedule.add_shift(Shift(name=f'EMS on {d.date()}',
                                description=f'Standby for EMS from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='EMS',
                                min_employees=1,
                                max_employees=1))
        # Observe
        schedule.add_shift(Shift(name=f'Observe on {d.date()}',
                                description=f'Observe from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='observe',
                                min_employees=1,
                                max_employees=1))
        # พอป การบิน (AMD)
        schedule.add_shift(Shift(name=f'AND on {d.date()}',
                                description=f'พอป. การบิน (Air medical director) from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='AMD',
                                min_employees=1,
                                max_employees=1))
        # แพทย์บิน (AVD)
        schedule.add_shift(Shift(name=f'AVD on {d.date()}',
                                description=f'แพทย์บิน (Aviation doctor) from 8.00 - 16.00',
                                duration=timedelta(hours=8),
                                start_time=datetime(d.year, d.month, d.day, 8, 0),
                                shift_type='AVD',
                                min_employees=1,
                                max_employees=1))


# remove employee
schedule.remove_employee(schedule.employees[0])


# remove shift
# schedule.remove_shift(schedule.shifts[0])

# [shift.date for shift in schedule.shifts]

# # display schedule
# schedule.display()

# # solve the schedule
# schedule.solve()

# schedule.display_table()


# schedule.show(format= "table", group_by='shift type')

# schedule.to_csv('/Users/spatipan/Library/CloudStorage/OneDrive-ChiangMaiUniversity/documents/shift_scheduler/data/outputs/schedule.csv')


{'assisted executive', 'assisted instructor', 'executive', 'instructor'}